In [1]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 10.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import openai
from tqdm import tqdm

In [3]:
df = pd.read_csv("nodes_with_examples_final.csv")

In [4]:
len(df)

1190

In [5]:
df.head()

,word,romaji,explanation,properties,examples
0,パチパチ,pachipachi,"crackling, cracking, snapping, popping, clicki...",popular; onomatopoeic or mimetic word; adverb;...,囲炉裏の側にいるとぱちぱちという音が聞こえてくる。; 日本には口にいれるとパチパチとはじける...
1,ハラハラ,harahara,"to feel anxious, to feel nervous, to be kept i...",popular; onomatopoeic or mimetic word; noun or...,偉い人が集まるパーティで自分が失礼なことしないかどうかハラハラする。; 紅葉の季節が終わって...
2,あっさり,assari,"easily, readily, quickly, flatly (refuse); lig...",popular; JLPT N1; onomatopoeic or mimetic word...,このラーメンはすごくあっさりした味。; 白大豆よりこの黒大豆で作られた味噌汁はあっさりしてい...
3,あべこべ,abekobe,"contrary, opposite, inverse, reverse, back-to-...",popular; JLPT N1; onomatopoeic or mimetic word...,NaN
4,あやふや,ayafuya,"uncertain, vague, ambiguous",popular; JLPT N1; onomatopoeic or mimetic word...,NaN


In [ ]:
GPT_APIKEY = input("Your GPT-3 API Key: ")
openai.api_key = GPT_APIKEY
EMBEDDING_MODEL = "text-embedding-ada-002"

In [7]:
def get_embedding(text, model=EMBEDDING_MODEL):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [8]:
df["embedding"] = df["explanation"].apply(lambda x: get_embedding(x))

In [9]:
df.to_csv("with_embedding.csv")

In [10]:
from openai.embeddings_utils import cosine_similarity

def search_synonyms(ddf, index, cutoff=0.9):
    query_embedding = ddf.iloc[index]["embedding"]
    res = ddf["embedding"].apply(lambda x: cosine_similarity(x, query_embedding)).reset_index(name="similarity")
    
    return res[res["similarity"] > cutoff]

In [11]:
nodes = []
edges = set()


for i, row in tqdm(df.iterrows()):
    
    nodes.append([i, row["word"], row["romaji"], row["explanation"], row["properties"], row["examples"]])


    result_index = search_synonyms(df, i)
    for j, r in result_index.iterrows():
        if i != r["index"]:
            min_tuple = int(min(i, r["index"]))
            max_tuple = int(max(i, r["index"]))
            edges.add(f"{min_tuple},{max_tuple},{round(r['similarity'], 5)}")

1190it [13:40,  1.45it/s]


In [12]:
node_df = pd.DataFrame(nodes, columns =['index', 'word', 'romaji', 'explanation', 'properties', 'examples'])

node_df.to_csv("nodes.csv", sep=",", index = False)

In [13]:
edges_content = ",".join(["from", "to", "similarity"]) + "\n"

for e in edges:
    edges_content += ",".join(e.split(",")) + "\n"

with open("edges.csv", "w") as output:
    output.write(edges_content)

In [15]:
node_df["examples"].isna()

0       False
1       False
2       False
3        True
4        True
        ...  
1185     True
1186     True
1187     True
1188     True
1189     True
Name: examples, Length: 1190, dtype: bool